# [LAB#04] 로지스틱 회귀 (logistic regression)

- 분석 데이터: "**제7차 한국인 인체치수 측정 데이터**"
- 로지스틱 회귀를 직접 구현 (logistic regression from scratch)


$\theta(t+1) = \theta(t) - \eta\frac{\partial NLL(\theta)}{\partial \theta}$
$\ \longrightarrow \ $ 
$\theta(t+1) = \theta(t) - \eta \cdot \left( -\!\sum_{i} ( y_{i} - \hat{y}_{i})X_{i} \right)$

In [3]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm  # pip install jupyter jupyterlab ipywidgets --upgrade

## 데이터 준비

In [4]:
df = pd.read_csv("2015_7th_korbody.csv",
                 thousands=',')

features = ["ⓞ_02_성별",
            "①_003_키",
            "①_031_몸무게",
            "①_104_손너비"]

sdf = df[features].copy()
sdf = sdf.sample(frac=1).reset_index(drop=True)  # Shuffle the data

sdf.columns = ["sex", "height", "weight", "hand_width"]

sdf.loc[sdf["sex"] == "남", "sex"] = 0
sdf.loc[sdf["sex"] == "여", "sex"] = 1

sdf.dropna(inplace=True)

In [5]:
sdf.head()

,sex,height,weight,hand_width
0,0,1800.0,94.5,76.0
1,0,1684.0,79.5,94.0
2,1,1699.0,71.6,76.0
3,0,1806.0,80.3,87.0
4,1,1510.0,55.0,77.0


## 데이터 나누기

In [6]:
iend_train = 3001
sdf_train = sdf.loc[:iend_train]
sdf_test = sdf.loc[iend_train:]

sdf_train = sdf_train.reset_index(drop=True) 
sdf_test = sdf_test.reset_index(drop=True)

In [7]:
sdf_train_male = sdf_train[sdf_train["sex"] == 0]
sdf_train_female = sdf_train[sdf_train["sex"] == 1]

sdf_test_male = sdf_test[sdf_test["sex"] == 0]
sdf_test_female = sdf_test[sdf_test["sex"] == 1]

In [8]:
print("[Train]")
print("- Num. People:", sdf_train.shape[0])
print("- Num. Males:", sdf_train_male.shape[0])
print("- Num. Females:", sdf_train_female.shape[0])

print("[Test]")
print("- Num. People:", sdf_test.shape[0])
print("- Num. Males:", sdf_test_male.shape[0])
print("- Num. Females:", sdf_test_female.shape[0])

[Train]
- Num. People: 2998
- Num. Males: 1461
- Num. Females: 1537
[Test]
- Num. People: 3415
- Num. Males: 1729
- Num. Females: 1686


In [9]:
sdf_train.iloc[:, 1:]

,height,weight,hand_width
0,1800.0,94.5,76.0
1,1684.0,79.5,94.0
2,1699.0,71.6,76.0
3,1806.0,80.3,87.0
4,1510.0,55.0,77.0
...,...,...,...
2993,1549.0,55.8,76.0
2994,1614.0,51.6,75.0
2995,1568.0,59.6,74.0
2996,1714.0,76.3,83.0


In [10]:
y_train = np.array(sdf_train.iloc[:, 0], dtype=np.float64)
x = sdf_train.iloc[:, 1:]
x = (x - x.mean()) / x.std()

X_train = np.column_stack([np.ones(sdf_train.shape[0]), x])

bhat = 0.001 * np.ones(4, dtype=np.float64)  # theta: parameters

In [11]:
yhat = 1/(1 + np.exp(-np.dot(X_train, bhat)))    
pred = yhat > 0.5
acc = (pred == y_train).sum() / y_train.shape[0]
print("[Before training] Acc:", acc)

[Before training] Acc: 0.14342895263509006


## 무작위 추출 (random sampling): 모델 훈련

In [13]:
num_epoch = 100000
bhat_min = None
acc_min = 0.0
loss_min = np.inf

for i in range(num_epoch):
    bhat = np.random.uniform(-1, 1, size=4)
    yhat = 1/(1 + np.exp(-np.dot(X_train, bhat)))     
    loss = -( np.mean(y_train * np.log(yhat) + (1 - y_train) * np.log(1 - yhat)) )  # NLL
    
    pred = yhat > 0.5
    acc = (pred == y_train).sum() / y_train.shape[0]
    
    if loss < loss_min:
        loss_min = loss
        acc_min = acc
        bhat_min = bhat.copy()            
        print("[Train #%04d] Loss(min)=%f, Acc(min)=%f"%(i+1, loss_min, acc_min))                
    # end of if


[Train #0001] Loss(min)=0.644175, Acc(min)=0.646765
[Train #0014] Loss(min)=0.496640, Acc(min)=0.776184
[Train #0025] Loss(min)=0.427352, Acc(min)=0.860240
[Train #0028] Loss(min)=0.383419, Acc(min)=0.855570
[Train #0055] Loss(min)=0.370693, Acc(min)=0.865243
[Train #0112] Loss(min)=0.338467, Acc(min)=0.884923
[Train #0223] Loss(min)=0.303910, Acc(min)=0.895597
[Train #1510] Loss(min)=0.302558, Acc(min)=0.881588
[Train #1760] Loss(min)=0.299730, Acc(min)=0.892929
[Train #7179] Loss(min)=0.295530, Acc(min)=0.887258
[Train #8536] Loss(min)=0.291166, Acc(min)=0.894263
[Train #13750] Loss(min)=0.290559, Acc(min)=0.890927


## 무작위 추출 (random sampling): 모델 테스트

In [14]:
y_test = np.array(sdf_test.iloc[:, 0], dtype=np.float64)
x = sdf_test.iloc[:, 1:]
x = (x - x.mean()) / x.std()

X_test = np.column_stack([np.ones(sdf_test.shape[0]), x])

In [15]:
bhat = bhat_min
yhat = 1/(1 + np.exp(-np.dot(X_test, bhat)))    
pred = yhat > 0.5
acc_test = (pred == y_test).sum() / y_test.shape[0]
print("[Test] Acc:", acc_test)

[Test] Acc: 0.8995607613469986


## 경사하강법 (gradient descent): 모델 훈련

In [16]:
bhat = np.random.uniform(-10, 10, size=4) #np.zeros(4, dtype=np.float64)
num_epoch = 10000
eta = 1e-4

In [18]:
for i in range(num_epoch):
    yhat = 1/(1 + np.exp(-np.dot(X_train, bhat)))    
    
    loss = -( np.mean(y_train * np.log(yhat) + (1 - y_train) * np.log(1 - yhat)) )
    bhat = bhat + eta * np.dot(y_train - yhat, X_train)  # Update the parameters
    
    pred = yhat > 0.5
    acc = (pred == y_train).sum() / y_train.shape[0]
    print("[Train #%04d] Loss=%f, Acc=%f"%(i+1, loss, acc))

[Train #0001] Loss=7.544469, Acc=0.203135
[Train #0002] Loss=7.191325, Acc=0.208472
[Train #0003] Loss=6.844763, Acc=0.215811
[Train #0004] Loss=6.505475, Acc=0.221815
[Train #0005] Loss=6.174245, Acc=0.230153
[Train #0006] Loss=5.851888, Acc=0.237825
[Train #0007] Loss=5.539175, Acc=0.249833
[Train #0008] Loss=5.236809, Acc=0.265177
[Train #0009] Loss=4.945448, Acc=0.277185
[Train #0010] Loss=4.665755, Acc=0.289193
[Train #0011] Loss=4.398370, Acc=0.306538
[Train #0012] Loss=4.143794, Acc=0.321214
[Train #0013] Loss=3.902398, Acc=0.337558
[Train #0014] Loss=3.674472, Acc=0.356905
[Train #0015] Loss=3.460153, Acc=0.375250
[Train #0016] Loss=3.259381, Acc=0.391261
[Train #0017] Loss=3.071941, Acc=0.408939
[Train #0018] Loss=2.897467, Acc=0.426284
[Train #0019] Loss=2.735470, Acc=0.442628
[Train #0020] Loss=2.585379, Acc=0.459640
[Train #0021] Loss=2.446583, Acc=0.472982
[Train #0022] Loss=2.318466, Acc=0.487658
[Train #0023] Loss=2.200413, Acc=0.504003
[Train #0024] Loss=2.091781, Acc=0

## 경사하강법 (gradient descent): 모델 테스트

In [19]:
y_test = np.array(sdf_test.iloc[:, 0], dtype=np.float64)
x = sdf_test.iloc[:, 1:]
x = (x - x.mean())/x.std()

X_test = np.column_stack([np.ones(sdf_test.shape[0]), x])

In [20]:
yhat = 1/(1 + np.exp(-np.dot(X_test, bhat)))    
pred = yhat > 0.5
acc_test = (pred == y_test).sum() / y_test.shape[0]
print("[Test] Acc:", acc_test)

[Test] Acc: 0.927086383601757
